In [1]:
import cv2
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 함수 목록

In [26]:
# 사람 정확도 뽑아내는 함수
def person_acc(data):
    if 'person: ' in data:
        return int(data.split('person: ')[-1].split('%')[0]) # 이해 안가면 여쭤봐주세요
    else: return 0

# fps 뽑아내는 함수
def fps(data):
    return float(data.split('FPS:')[-1])

    
def parsing_log(log_dir, accuracy):
    with open(log_dir, 'r') as f:
        string=f.read()
    
    p=re.compile(r'\n+')
    cleaned_string = p.sub('', string)

    cleaned_string = cleaned_string.replace('Stream closed. cvWriteFrame input video stream closed.  closing... closed!output_video_writer closed.', '')

    content = cleaned_string.split(' cvWriteFrame Objects:')[1:] # 반복되는 단어 기준으로 스트링을 쪼갭니다.
    # 0번이 아니라 1번부터 부르는 이유는 0번에 필요없는 문자가 들어가서

    length = len(content)

    # 쪼갠 내용을 데이터프레임으로 바꿈

    df = pd.DataFrame(content)

    df['acc'] = df[0].apply(person_acc)
    df['fps'] = df[0].apply(fps)

    # df.loc[df['acc']!=0, :] 이 코드로 잘 구분되었는지 확인 가능

    # 행 추가(앞에 두개 뒤에 한개)
    df.index= (range(2, length+2))
    df.loc[0, :] = [0, 0, 0]
    df.loc[1, :] = [0, 0, 0]
    df.loc[length+2, :] = [0, 0, 0]
    df = df.sort_index(axis=0)

    return list((np.rint((df.loc[df['acc']>=accuracy, :].index).astype(float)/(1.25))).astype(int))

# 특정 정확도 이상의 프레임 추출
def frame_extracting_by_log(video_dir, log):
    i= 0 # 파일 명의 인덱스를 만들어 주기 위한 변수
    result = []
    capture = cv2.VideoCapture(video_dir)
    
    while True:

        ret, frame = capture.read()

        if log.count(i):
            cv2.imwrite(f'result/result{i}.png', frame)
            result.append(i)

        if i != capture.get(cv2.CAP_PROP_FRAME_COUNT):
            i+=1

        else : return result
            
def frame_extracting_by_color(video_dir) :
    lower_purple = np.array([149,254,252]) # Min_range
    upper_purple = np.array([150,255,255]) # Max_range
    i= 0 # 파일 명의 인덱스를 만들어 주기 위한 변수
    result = []
    capture = cv2.VideoCapture(video_dir)
    
    while True :
        ret, frame = capture.read() # 비디오 읽어오기
            
        img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) # BGR 이미지를 HSV로 바꿈.
        image_mask = cv2.inRange(img_hsv, lower_purple, upper_purple) # lower - upper 지정한 범위의 색이 있는지 확인하고, 나머지 부분을 모드 0으로 채움.
        img_result = cv2.bitwise_and(frame, frame, mask = image_mask) # 
        
        if img_result.sum() > np.zeros((img_result.shape)).sum() + 200000 : # 같은 것이 없다면, 그 모든 배열의 합은 0일 것이고 0이 아니라면 최소 하나의 색상이 있는것과 같다.
            cv2.imwrite(f'result/result{i}.png', frame)
            result.append(i)

        if i != capture.get(cv2.CAP_PROP_FRAME_COUNT):
            i+=1

        else : return result
        
def face_detect(result):
    face_cascade = cv2.CascadeClassifier('./face_detect/data/haarcascades/haarcascade_frontalface_default.xml')
    image_list = [f'result/result{i}.png' for i in result]
    imgNum = 0
    
    for image in image_list:
        image=cv2.imread(image)
        grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(grayImage, 1.03, 5)
             
        if len(faces)!=0:
              
            for (x,y,w,h) in faces:
                # cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
                cropped = image[y - int(h / 4):y + h + int(h / 4), x - int(w / 4):x + w + int(w / 4)]
                # 이미지를 저장
                height, width, channel = cropped.shape
                img_result = cv2.pyrUp(cropped, (8*width, 8*height), borderType=cv2.BORDER_DEFAULT)
                cv2.imwrite(f"face/face{imgNum}.png", img_result) # cropped , img_result
                imgNum += 1
                
def image_cropping(image, h1, h2, w1, w2):
    return image[h1:h2, w1:w2]

In [27]:
over_100 = parsing_log('data/test_det_log.txt', 100)

result = frame_extracting_by_log('data/test_det.mp4', over_100)

face_detect(result)